In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt

In [2]:
n = 100
c_dim = 4
x_dim = 2
y_dim = 3
C = torch.rand((n, c_dim)) - .5 
W_1 = C.sum(axis=1).unsqueeze(-1) ** 2
W_2 = - C.sum(axis=1).unsqueeze(-1)
b_1 = C[:, 0].unsqueeze(-1)
b_2 = C[:, 1].unsqueeze(-1)
W_full = torch.cat((W_1, W_2), axis=1)
b_full = b_1 + b_2
X = torch.rand((n, x_dim)) - .5
Y_1 = X[:, 0].unsqueeze(-1) * W_1 + b_1
Y_2 = X[:, 1].unsqueeze(-1) * W_2 + b_2
Y_3 = X.sum(axis=1).unsqueeze(-1)
Y = torch.cat((Y_1, Y_2, Y_3), axis=1)
C.shape, X.shape, Y.shape

(torch.Size([100, 4]), torch.Size([100, 2]), torch.Size([100, 3]))

In [73]:
n = 20
w = torch.nn.functional.softmax(torch.rand((n, 3)), dim=1)
u = torch.nn.functional.softmax(torch.rand((n, 4)), dim=1)
v = torch.nn.functional.softmax(torch.rand((n, 5)), dim=1)
A = torch.rand((7, 5, 4, 3))

In [75]:
b = A.unsqueeze(0).expand(v.shape[0], -1, -1, -1, -1).unsqueeze(1)
print(b.shape)
w_batch = w.unsqueeze(-1).unsqueeze(1).unsqueeze(1).unsqueeze(1)
print(w_batch.shape)
print()
b = torch.matmul(b, w_batch).squeeze(-1)
print(b.shape)
u_batch = u.unsqueeze(-1).unsqueeze(1).unsqueeze(1)
print(u_batch.shape)
print()
b = torch.matmul(b, u_batch).squeeze(-1)
print(b.shape)
v_batch = v.unsqueeze(-1).unsqueeze(1)
print(v_batch.shape)
print()
b = torch.matmul(b, v_batch).squeeze(-1)
print(b.shape)

torch.Size([20, 1, 7, 5, 4, 3])
torch.Size([20, 1, 1, 1, 3, 1])

torch.Size([20, 1, 7, 5, 4])
torch.Size([20, 1, 1, 4, 1])

torch.Size([20, 1, 7, 5])
torch.Size([20, 1, 5, 1])

torch.Size([20, 1, 7])


In [86]:
n = 20
w = torch.nn.functional.softmax(torch.rand((n, 5)), dim=1)
u = torch.nn.functional.softmax(torch.rand((n, 4)), dim=1)
v = torch.nn.functional.softmax(torch.rand((n, 3)), dim=1)
batch_weights = [w, u, v]

archetypes = torch.rand((7, 49, 5, 4, 3))
# first dim is the batch size
# last d dims are the catcols
# middle n-d-1 dims are the archetypes

In [89]:
batch_size = batch_weights[0].shape[0]
expand_dims = [batch_size] + [-1 for _ in range(len(archetypes.shape))]
batch_archetypes = archetypes.unsqueeze(0).expand(expand_dims)
d = len(batch_weights)
for batch_w in batch_weights[::-1]:
    batch_w = batch_w.unsqueeze(-1)
    d = len(batch_archetypes.shape) - len(batch_w.shape)
    for _ in range(d):
        batch_w = batch_w.unsqueeze(1)
    print(batch_archetypes.shape)
    print(batch_w.shape)
    batch_archetypes = torch.matmul(batch_archetypes, batch_w).squeeze(-1)
    print()
print(batch_archetypes.shape)

torch.Size([20, 7, 49, 5, 4, 3])
torch.Size([20, 1, 1, 1, 3, 1])

torch.Size([20, 7, 49, 5, 4])
torch.Size([20, 1, 1, 4, 1])

torch.Size([20, 7, 49, 5])
torch.Size([20, 1, 5, 1])

torch.Size([20, 7, 49])


In [91]:
import torch.nn as nn
class CatCol(nn.Module):
    def __init__(self, in_dims, out_shape):
        super(CatCol, self).__init__()
        init_mat = torch.rand(list(out_shape) + list(in_dims)) * 2e-2 - 1e-2
        self.archetypes = nn.parameter.Parameter(init_mat, requires_grad=True)

    def forward(self, *batch_weights):
        batch_size = batch_weights[0].shape[0]
        expand_dims = [batch_size] + [-1 for _ in range(len(self.archetypes.shape))]
        batch_archetypes = self.archetypes.unsqueeze(0).expand(expand_dims)
        for batch_w in batch_weights[::-1]:
            batch_w = batch_w.unsqueeze(-1)
            d = len(batch_archetypes.shape) - len(batch_w.shape)
            for _ in range(d):
                batch_w = batch_w.unsqueeze(1)
            batch_archetypes = torch.matmul(batch_archetypes, batch_w).squeeze(-1)
        return batch_archetypes

In [99]:
w = torch.nn.functional.softmax(torch.ones((n, 5)), dim=1)
u = torch.nn.functional.softmax(torch.ones((n, 4)), dim=1)
v = torch.nn.functional.softmax(torch.ones((n, 3)), dim=1)
catcol = CatCol((5, 4, 3), (7, 8))
batch_archetypes = catcol(w, u, v)

torch.Size([20, 7, 8, 5, 4, 3])
torch.Size([20, 1, 1, 1, 3, 1])

torch.Size([20, 7, 8, 5, 4])
torch.Size([20, 1, 1, 4, 1])

torch.Size([20, 7, 8, 5])
torch.Size([20, 1, 5, 1])

torch.Size([20, 7, 8])


In [100]:
batch_archetypes[0] == batch_archetypes[1]

tensor([[True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True]])

In [122]:
class NGAM(nn.Module):
    def __init__(self, input_dim, output_dim, width=25, layers=2, activation=nn.ReLU):
        super(NGAM, self).__init__()
        self.intput_dim = input_dim
        self.output_dim = output_dim
        hidden_layers = lambda: [layer for _ in range(0, layers - 2) for layer in (nn.Linear(width, width), activation())]
        nam_layers = lambda: [nn.Linear(1, width), activation()] + hidden_layers() + [nn.Linear(width, output_dim)]
        self.nams = [nn.Sequential(*nam_layers()) for _ in range(input_dim)]

    def forward(self, x):
        batch_size = x.shape[0]
        ret = torch.zeros((batch_size, self.output_dim))
        for i, nam in enumerate(self.nams):
            ret += nam(x[:, i].unsqueeze(-1))
        return ret

In [3]:
n = 100
c_dim = 4
x_dim = 2
y_dim = 3
C = torch.rand((n, c_dim)) - .5 
W_1 = C.sum(axis=1).unsqueeze(-1) ** 2
W_2 = - C.sum(axis=1).unsqueeze(-1)
b_1 = C[:, 0].unsqueeze(-1)
b_2 = C[:, 1].unsqueeze(-1)
W_full = torch.cat((W_1, W_2), axis=1)
b_full = b_1 + b_2
X = torch.rand((n, x_dim)) - .5
Y_1 = X[:, 0].unsqueeze(-1) * W_1 + b_1
Y_2 = X[:, 1].unsqueeze(-1) * W_2 + b_2
Y_3 = X.sum(axis=1).unsqueeze(-1)
Y = torch.cat((Y_1, Y_2, Y_3), axis=1)
C.shape, X.shape, Y.shape

(torch.Size([100, 4]), torch.Size([100, 2]), torch.Size([100, 3]))

In [56]:
import time
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.parameter import Parameter


def MSE(beta, mu, x, y):
    y_hat = (beta * x).sum(axis=1).unsqueeze(-1) + mu
    residual = y_hat - y
    return residual.pow(2).mean()


class SoftSelect(nn.Module):
    """
    Parameter sharing for multiple context encoders:
    Batched computation for mapping many subtypes onto d-dimensional archetypes
    """
    def __init__(self, in_dims, out_shape):
        super(SoftSelect, self).__init__()
        init_mat = torch.rand(list(out_shape) + list(in_dims)) * 2e-2 - 1e-2
        self.archetypes = nn.parameter.Parameter(init_mat, requires_grad=True)

    def forward(self, *batch_weights):
        batch_size = batch_weights[0].shape[0]
        expand_dims = [batch_size] + [-1 for _ in range(len(self.archetypes.shape))]
        batch_archetypes = self.archetypes.unsqueeze(0).expand(expand_dims)
        for batch_w in batch_weights[::-1]:
            batch_w = batch_w.unsqueeze(-1)
            d = len(batch_archetypes.shape) - len(batch_w.shape)
            for _ in range(d):
                batch_w = batch_w.unsqueeze(1)
            batch_archetypes = torch.matmul(batch_archetypes, batch_w).squeeze(-1)
        return batch_archetypes


class NGAM(nn.Module):
    """
    Generalized additive model implemented with neural networks for feature-specific functions
    """
    def __init__(self, input_dim, output_dim, width=25, layers=2, activation=nn.ReLU):
        super(NGAM, self).__init__()
        self.intput_dim = input_dim
        self.output_dim = output_dim
        hidden_layers = lambda: [layer for _ in range(0, layers - 2) for layer in (nn.Linear(width, width), activation())]
        nam_layers = lambda: [nn.Linear(1, width), activation()] + hidden_layers() + [nn.Linear(width, output_dim)]
        self.nams = [nn.Sequential(*nam_layers()) for _ in range(input_dim)]

    def forward(self, x):
        batch_size = x.shape[0]
        ret = torch.zeros((batch_size, self.output_dim))
        for i, nam in enumerate(self.nams):
            ret += nam(x[:, i].unsqueeze(-1))
        return ret

    
class ContextualizedRegressionModule(nn.Module):
    """
    Estimates the weights and offset in a context-specific and task-specific regression of X onto Y
    """
    def __init__(self, context_dim, task_dim, beta_dim=1, context_archetypes=10, task_archetypes=10, nam_width=25, nam_layers=2, activation=nn.ReLU):
        super(ContextualizedRegressionModule, self).__init__()
        self.context_dim = context_dim
        self.task_dim = task_dim
        self.beta_dim = beta_dim

        self.context_encoder = NGAM(context_dim, context_archetypes, width=nam_width, layers=nam_layers)
        self.task_encoder = NGAM(task_dim, task_archetypes, width=nam_width, layers=nam_layers)
        self.softselect = SoftSelect((context_archetypes, task_archetypes), (beta_dim + 1, ))

    def forward(self, c, t):
        Z_context = self.context_encoder(c)
        Z_task = self.task_encoder(t)
        W = self.softselect(F.softmax(Z_context, dim=1), F.softmax(Z_task, dim=1))
        beta = W[:, :-1]
        mu = W[:, -1:]
        return beta, mu
    

class UnivariateDataset:
    def __init__(self, C, X, Y, batch_size=1, dtype=torch.float, device=torch.device('cpu')):
        """
        C: (n x c_dim)
        X: (n x x_dim)
        Y: (n x y_dim)
        """
        self.C = torch.tensor(C, dtype=dtype, device=device)
        self.X = torch.tensor(X, dtype=dtype, device=device)
        self.Y = torch.tensor(Y, dtype=dtype, device=device)
        self.n_i = 0
        self.x_i = 0
        self.y_i = 0
        self.n = C.shape[0]
        self.c_dim = C.shape[-1]
        self.x_dim = X.shape[-1]
        self.y_dim = Y.shape[-1]
        self.task_dim = self.x_dim + self.y_dim
        self.batch_size = batch_size
        self.dtype = dtype
        self.device = device

    def __iter__(self):
        self.n_i = 0
        self.x_i = 0
        self.y_i = 0
        return self
    
    def sample(self):
        t = torch.zeros(self.task_dim)
        t[self.x_i] = 1
        t[self.x_dim + self.y_i] = 1
        ret = (
            self.C[self.n_i].unsqueeze(0),
            t.unsqueeze(0),
            self.X[self.n_i, self.x_i:self.x_i+1].unsqueeze(0),
            self.Y[self.n_i, self.y_i:self.y_i+1].unsqueeze(0),
        )
        self.y_i += 1
        if self.y_i >= self.y_dim:
            self.x_i += 1
            self.y_i = 0
        if self.x_i >= self.x_dim:
            self.n_i += 1
            self.x_i = 0
        return ret

    def __next__(self):
        """
        Returns a batch_size paired sample (c, t, x, y)
        If there are fewer than batch_size samples remaining, returns n - batch_size samples
        c: (batch_size, c_dim)
        t: (batch_size, task_dim * 2)  [x_task y_task]
        x: (batch_size, 1)
        y: (batch_size, 1)
        """
        if self.n_i >= self.n:
            self.n_i = 0
            raise StopIteration
        C_batch, T_batch, X_batch, Y_batch = self.sample()
        while len(C_batch) < self.batch_size and self.n_i < self.n:
            C_s, T_s, X_s, Y_s = self.sample()
            C_batch = torch.cat((C_batch, C_s))
            T_batch = torch.cat((T_batch, T_s))
            X_batch = torch.cat((X_batch, X_s))
            Y_batch = torch.cat((Y_batch, Y_s))
        return C_batch, T_batch, X_batch, Y_batch
    
    def __len__(self):
        return self.n * self.x_dim * self.y_dim
    

class ContextualizedUnivariateRegression:
    def __init__(self, context_dim, x_dim, y_dim, context_archetypes=10, task_archetypes=10, 
                l1=0, encoder_width=25, encoder_layers=2, activation=nn.ReLU, bootstraps=None, 
                device=torch.device('cpu')):
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.taskpair_dim = x_dim + y_dim
        module_params = {
            'context_dim': context_dim,
            'task_dim': self.taskpair_dim,
            'beta_dim': 1,
            'context_archetypes': context_archetypes,
            'task_archetypes': task_archetypes,
            'nam_width': encoder_width,
            'nam_layers': encoder_layers,
            'activation': activation,
        }
        if bootstraps is None:
            self.model = ContextualizedRegressionModule(**module_params)
            self.models = [self.model]
        else:
            self.model = None
            self.models = [ContextualizedRegressionModule(**module_params) for _ in range(bootstraps)]
        self.to(device)

    def to(self, device):
        self.device = device
        for model in self.models:
            model.to(self.device)
        return self

    def _loss(self, outputs, X, Y):
        beta, mu = outputs
        mse = MSE(beta, mu, X, Y)
        return mse

    def _fit(self, model, C, X, Y, epochs, batch_size, optimizer=torch.optim.Adam, lr=1e-3, 
             validation_set=None, es_patience=np.inf, es_epoch=0, silent=False):
        model.train()
        device = next(model.parameters()).device
        opt = optimizer(model.parameters(), lr=lr)
        dataset = UnivariateDataset(C, X, Y, batch_size=batch_size, dtype=torch.float, device=device)
        val_dataset = None
        if validation_set is not None:
            val_dataset = UnivariateDataset(*validation_set, batch_size=batch_size, dtype=torch.float, device=device)
        progress_bar = tqdm(range(epochs), disable=silent)
        min_loss = np.inf  # for early stopping
        es_count = 0
        for epoch in progress_bar:
            for batch_i, (c, t, x, y) in enumerate(dataset):
                loss = None
                # Train
                outputs = model(c, t)
                loss = self._loss(outputs, x, y)
                opt.zero_grad()
                loss.backward()
                opt.step()
                # Update UI and check validation set for early stopping
                train_desc = f'[Train MSE: {loss.item():.4f}] [Sample: {batch_size * batch_i}/{len(dataset)}] Epoch'
                if val_dataset is not None:
                    val_loss_batches = []
                    for c_val, t_val, x_val, y_val in val_dataset:
                        val_outputs = model(c_val, t_val)
                        val_loss_batch = self._loss(val_outputs, x_val, y_val).item()
                        val_loss_batches.append(val_loss_batch)
                    val_loss = np.mean(val_loss_batches)
                    train_desc = f"[Val MSE: {val_loss:.4f}] " + train_desc
                    if epoch >= es_epoch:
                        if val_loss < min_loss:
                            min_loss = val_loss
                            es_count = 0
                        else:
                            es_count += 1
                progress_bar.set_description(train_desc)
                if es_count > es_patience:
                    model.eval()
                    return
        model.eval()

    def fit(self, C, X, Y, epochs, batch_size, optimizer=torch.optim.Adam, lr=1e-3, validation_set=None, es_patience=np.inf, es_epoch=0, silent=False):
        fit_params = {
            'C': C, 'X': X, 'Y': Y, 'epochs': epochs, 'batch_size': batch_size,
            'optimizer': optimizer, 'lr': lr,
            'validation_set': validation_set, 'es_epoch': es_epoch, 'es_patience': es_patience,
            'silent': silent,
        }
        if self.model:
            fit_params['model'] = self.model
            self._fit(**fit_params)
        else:
            for model in self.models:
                boot_idx = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
                fit_params.update({
                    'model': model, 'C': C[boot_idx], 'X': X[boot_idx], 'Y': Y[boot_idx],
                })
                self._fit(**fit_params)

    def _predict_coefs(self, model, C):
        """
        Predict a (1, c_dim, x_dim, y_dim) matrix of regression coefficients and offsets for each context
        beta[i,j] and mu[i,j] solve the regression problem y_j = beta[i,j] * x_i + mu[i,j]
        returns a numpy matrix
        """
        n = C.shape[0]
        betas = torch.zeros((n, self.x_dim, self.y_dim))
        mus = torch.zeros((n, self.x_dim, self.y_dim))
        for i in range(n):  # Predict per-sample to avoid OOM
            C_i = torch.Tensor(C[i])
            for t_x in range(self.x_dim):
                for t_y in range(self.y_dim):
                    task = torch.zeros(self.taskpair_dim)
                    task[t_x] = 1
                    task[self.x_dim + t_y] = 1
                    beta, mu = model(C_i.unsqueeze(0), task.unsqueeze(0))
                    beta, mu = beta.cpu().detach(), mu.cpu().detach()
                    betas[i, t_x, t_y] = beta.squeeze()
                    mus[i, t_x, t_y] = mu.squeeze()
        return betas.unsqueeze(0).numpy(), mus.unsqueeze(0).numpy()

    def predict_coefs(self, C, all_bootstraps=False):
        """
        Predict an (x_dim, y_dim) matrix of regression coefficients for each context
        Returns a numpy matrix (1 or bootstraps, n, x_dim, y_dim)
        """
        betas, mus = self._predict_coefs(self.models[0], C)
        for model in self.models[1:]:
            betas_i, mus_i = self._predict_coefs(model, C)
            betas = np.concatenate((betas, betas_i), axis=0)
            mus = np.concatenate((mus, mus_i), axis=0)
        if all_bootstraps:
            return betas, mus
        return betas.mean(axis=0), mus.mean(axis=0)

    def predict_correlation(self, C, all_bootstraps=False):
        """
        Requires x_dim == y_dim
        Predict an (x_dim, y_dim) matrix of squared Pearson's correlation coefficients for each context
        Returns a numpy matrix (n, x_dim, y_dim, 1 or bootstraps)
        """
        betas, mus = self.predict_coefs(C, all_bootstraps=True)
        betas_T = np.transpose(betas, (0, 1, 3, 2))
        rho = betas * betas_T
        if all_bootstraps:
            return rho
        return rho.mean(axis=0)

    def get_mse(self, C, X, Y, all_bootstraps=False):
        """
        Returns the MSE of the model on a dataset
        Returns a numpy array (1 or bootstraps, )
        """
        dataset = UnivariateDataset(C, X, Y, batch_size=1, device=self.device)
        mses = np.zeros(len(self.models))
        for i, model in enumerate(self.models):
            for c, t, x, y in dataset:
                betas, mus = model(c, t)
                mse = MSE(betas, mus, x, y).item()
                mses[i] += 1 / len(dataset) * mse
        if not all_bootstraps:
            return mses.mean()
        return mses


class MultivariateDataset:
    def __init__(self, C, X, Y, batch_size=1, dtype=torch.float, device=torch.device('cpu')):
        """
        C: (n x c_dim)
        X: (n x x_dim)
        Y: (n x y_dim)
        """
        self.C = torch.tensor(C, dtype=dtype, device=device)
        self.X = torch.tensor(X, dtype=dtype, device=device)
        self.Y = torch.tensor(Y, dtype=dtype, device=device)
        self.n_i = 0
        self.y_i = 0
        self.n = C.shape[0]
        self.c_dim = C.shape[-1]
        self.x_dim = X.shape[-1]
        self.y_dim = Y.shape[-1]
        self.task_dim = self.y_dim
        self.batch_size = batch_size
        self.dtype = dtype
        self.device = device
        
    def __iter__(self):
        self.n_i = 0
        self.y_i = 0
        return self
        
    def sample(self):
        t = torch.zeros(self.task_dim)
        t[self.y_i] = 1
        ret = (
            self.C[self.n_i].unsqueeze(0),
            t.unsqueeze(0),
            self.X[self.n_i].unsqueeze(0),
            self.Y[self.n_i, self.y_i:self.y_i+1].unsqueeze(0),
        )
        self.y_i += 1
        if self.y_i >= self.y_dim:
            self.n_i += 1
            self.y_i = 0
        return ret
    
    def __next__(self):
        """
        Returns a batch_size sample (c, t, x, y)
        If there are fewer than batch_size samples remaining, returns n - batch_size samples
        c: (batch_size, c_dim)
        t: (batch_size, task_dim * 2)  [x_task y_task]
        x: (batch_size, x_dim)
        y: (batch_size, 1)
        """
        if self.n_i >= self.n:
            self.n_i = 0
            raise StopIteration
        C_batch, T_batch, X_batch, Y_batch = self.sample()
        while len(C_batch) < self.batch_size and self.n_i < self.n:
            C_s, T_s, X_s, Y_s = self.sample()
            C_batch = torch.cat((C_batch, C_s))
            T_batch = torch.cat((T_batch, T_s))
            X_batch = torch.cat((X_batch, X_s))
            Y_batch = torch.cat((Y_batch, Y_s))
        return C_batch, T_batch, X_batch, Y_batch
    
    def __len__(self):
        return self.n * self.y_dim


class ContextualizedRegression:
    def __init__(self, context_dim, x_dim, y_dim, context_archetypes=10, task_archetypes=10, 
                l1=0, encoder_width=25, encoder_layers=2, activation=nn.ReLU, bootstraps=None, 
                device=torch.device('cpu')):
        self.context_dim = context_dim
        self.x_dim = x_dim
        self.y_dim = y_dim
        self.task_dim = y_dim
        module_params = {
            'context_dim': context_dim,
            'task_dim': y_dim,
            'beta_dim': x_dim,
            'context_archetypes': context_archetypes,
            'task_archetypes': task_archetypes,
            'nam_width': encoder_width,
            'nam_layers': encoder_layers,
            'activation': activation,
        }
        if bootstraps is None:
            self.model = ContextualizedRegressionModule(**module_params)
            self.models = [self.model]
        else:
            self.model = None
            self.models = [ContextualizedRegressionModule(**module_params) for _ in range(bootstraps)]
        self.to(device)

    def to(self, device):
        self.device = device
        for model in self.models:
            model.to(self.device)
        return self

    def _loss(self, outputs, X, Y):
        beta, mu = outputs
        mse = MSE(beta, mu, X, Y)
        return mse

    def _fit(self, model, C, X, Y, epochs, batch_size, optimizer=torch.optim.Adam, lr=1e-3, 
             validation_set=None, es_patience=np.inf, es_epoch=0, silent=False):
        model.train()
        device = next(model.parameters()).device
        opt = optimizer(model.parameters(), lr=lr)
        dataset = MultivariateDataset(C, X, Y, batch_size=batch_size, dtype=torch.float, device=device)
        val_dataset = None
        if validation_set is not None:
            val_dataset = MultivariateDataset(*validation_set, batch_size=batch_size, dtype=torch.float, device=device)
        progress_bar = tqdm(range(epochs), disable=silent)
        min_loss = np.inf  # for early stopping
        es_count = 0
        for epoch in progress_bar:
            for batch_i, (c, t, x, y) in enumerate(dataset):
                loss = None
                # Train
                outputs = model(c, t)
                loss = self._loss(outputs, x, y)
                opt.zero_grad()
                loss.backward()
                opt.step()
                # Update UI and check validation set for early stopping
                train_desc = f'[Train MSE: {loss.item():.4f}] [Sample: {batch_size * batch_i}/{len(dataset)}] Epoch'
                if val_dataset is not None:
                    val_loss_batches = []
                    for c_val, t_val, x_val, y_val in val_dataset:
                        val_outputs = model(c_val, t_val)
                        val_loss_batch = self._loss(val_outputs, x_val, y_val).item()
                        val_loss_batches.append(val_loss_batch)
                    val_loss = np.mean(val_loss_batches)
                    train_desc = f"[Val MSE: {val_loss:.4f}] " + train_desc
                    if epoch >= es_epoch:
                        if val_loss < min_loss:
                            min_loss = val_loss
                            es_count = 0
                        else:
                            es_count += 1
                progress_bar.set_description(train_desc)
                if es_count > es_patience:
                    model.eval()
                    return
        model.eval()

    def fit(self, C, X, Y, epochs, batch_size, optimizer=torch.optim.Adam, lr=1e-3, validation_set=None, es_patience=np.inf, es_epoch=0, silent=False):
        fit_params = {
            'C': C, 'X': X, 'Y': Y, 'epochs': epochs, 'batch_size': batch_size,
            'optimizer': optimizer, 'lr': lr,
            'validation_set': validation_set, 'es_epoch': es_epoch, 'es_patience': es_patience,
            'silent': silent,
        }
        if self.model:
            fit_params['model'] = self.model
            self._fit(**fit_params)
        else:
            for model in self.models:
                boot_idx = np.random.choice(np.arange(len(X)), size=len(X), replace=True)
                fit_params.update({
                    'model': model, 'C': C[boot_idx], 'X': X[boot_idx], 'Y': Y[boot_idx],
                })
                self._fit(**fit_params)

    def _predict_coefs(self, model, C):
        """
        Predict a (1, c_dim, x_dim, y_dim) matrix of regression coefficients and offsets for each context
        beta[i,j] and mu[i,j] solve the regression problem y_j = beta[i,j] * x_i + mu[i,j]
        returns a numpy matrix
        """
        n = C.shape[0]
        betas = torch.zeros((n, self.x_dim, self.y_dim))
        mus = torch.zeros((n, self.y_dim))
        for i in range(n):  # Predict per-sample to avoid OOM
            C_i = torch.Tensor(C[i])
            for t_y in range(self.y_dim):
                task = torch.zeros(self.y_dim)
                task[t_y] = 1
                beta, mu = model(C_i.unsqueeze(0), task.unsqueeze(0))
                beta, mu = beta.cpu().detach(), mu.cpu().detach()
                betas[i, :, t_y] = beta.squeeze()
                mus[i, t_y] = mu.squeeze()
        return betas.unsqueeze(0).numpy(), mus.unsqueeze(0).numpy()

    def predict_coefs(self, C, all_bootstraps=False):
        """
        Predict an (x_dim, y_dim) matrix of regression coefficients for each context
        Returns a numpy matrix (1 or bootstraps, n, x_dim, y_dim)
        """
        betas, mus = self._predict_coefs(self.models[0], C)
        for model in self.models[1:]:
            betas_i, mus_i = self._predict_coefs(model, C)
            betas = np.concatenate((betas, betas_i), axis=0)
            mus = np.concatenate((mus, mus_i), axis=0)
        if all_bootstraps:
            return betas, mus
        return betas.mean(axis=0), mus.mean(axis=0)
    
    def predict_y(self, C, X, all_bootstraps=False):
        betas, mus = self.predict_coefs(C, all_bootstraps=True)
        y_hat = np.zeros((len(self.models), len(C), self.y_dim))
        for i in range(len(self.models)):
            for y_i in range(self.y_dim):
                beta_y = betas[i, :, :, y_i]
                mu_y = mus[i, :, y_i]
                y_hat[i, :, y_i] = (beta_y * X).sum(axis=1) + mu_y
        if all_bootstraps:
            return y_hat
        return y_hat.mean(axis=0)

    def get_mse(self, C, X, Y, all_bootstraps=False):
        """
        Returns the MSE of the model on a dataset
        Returns a numpy array (1 or bootstraps, )
        """
        dataset = MultivariateDataset(C, X, Y, batch_size=1, device=self.device)
        mses = np.zeros(len(self.models))
        for i, model in enumerate(self.models):
            for c, t, x, y in dataset:
                betas, mus = model(c, t)
                mse = MSE(betas, mus, x, y).item()
                mses[i] += 1 / len(dataset) * mse
        if not all_bootstraps:
            return mses.mean()
        return mses

In [57]:
# from dataset import to_pairwise
# C_pairwise, T_pairwise, X_pairwise, Y_pairwise = to_pairwise(C, X, Y)
# uds = MultivariateDataset(C, X, Y)
# count = 0
# for dp in uds:
#     print(dp)
#     count += 1
#     if count > 10:
#         break
# print(next(uds))
# model = ContextualizedRegressionModule(C_pairwise.shape[-1], T_pairwise.shape[-1])
# dataset = UnivariateDataset(C, X, Y, batch_size=1)
# model = ContextualizedRegressionModule(dataset.c_dim, dataset.task_dim,) # beta_dim=dataset.x_dim)
# opt = torch.optim.Adam(model.parameters(), lr=1e-2)
# for epoch in range(1):
#     for c, t, x, y in dataset:
#         beta, mu = model(c, t)
#         loss = MSE(beta, mu, x, y)  # this loop w unsqueeze is a hacky fix for the todo above
#         opt.zero_grad()
#         loss.backward()
#         opt.step()
#     loss_sum = 0
#     for c, t, x, y in dataset:
#         beta, mu = model(c, t)
#         loss = MSE(beta, mu, x, y).detach().item()  # this loop w unsqueeze is a hacky fix for the todo above
#         loss_sum += loss
#     print(loss_sum)
    
# model = ContextualizedRegressionModule(C_pairwise.shape[-1], T_pairwise.shape[-1])
# torch.stack((C[0], C[0], C[80])).shape
print(C.shape[-1], X.shape[-1], Y.shape[-1])
model = ContextualizedRegression(C.shape[-1], X.shape[-1], Y.shape[-1], bootstraps=2)
# model.fit(C, X, Y, epochs=10, batch_size=1, lr=1e-2)
# TODO: Write univariate predict fns, add multivariate contextualized regression

4 2 3


In [58]:
betas, mus = model.predict_coefs(C)
y_hat = model.predict_y(C.numpy(), X.numpy())
print(betas.shape, mus.shape, y_hat.shape)
model.get_mse(C, X, Y)

(100, 2, 3) (100, 3) (100, 3)


/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_1526/205546479.py:325: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.C = torch.tensor(C, dtype=dtype, device=device)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_1526/205546479.py:326: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.X = torch.tensor(X, dtype=dtype, device=device)
/var/folders/_6/468mk1cx3cb080z2r5gq0k_r0000gn/T/ipykernel_1526/205546479.py:327: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.Y = torch.tensor(Y, dtype=dtype, device=device)


0.1312595792254387

In [174]:
opt = torch.optim.Adam(model.parameters(), lr=1e-2)
for epoch in range(5000):
    beta, mu = model(C_pairwise, T_pairwise)
    loss = MSE(beta.squeeze(), mu, X_pairwise, Y_pairwise)  # this loop w unsqueeze is a hacky fix for the todo above
    opt.zero_grad()
    loss.backward()
    opt.step()
    if epoch % 100 == 0:
        print(loss.item())

0.10504741966724396
0.08994530886411667
0.07826512306928635
0.06986884772777557
0.0639653280377388
0.05981753394007683
0.056846413761377335
0.05463786423206329
0.05291229113936424
0.05148674547672272
0.05024436488747597
0.04911275953054428
0.04804883152246475
0.047028183937072754
0.046037837862968445
0.04507139325141907
0.04412606358528137
0.04320095479488373
0.04229608178138733
0.041411858052015305
0.04054883494973183
0.03970754146575928
0.03888847678899765
0.03809204697608948
0.037318550050258636
0.03656821325421333
0.035841163247823715
0.03513743355870247
0.034456998109817505
0.03379973769187927
0.03316546976566315
0.03255394101142883
0.031964849680662155
0.03139783442020416
0.030852477997541428
0.03032832406461239
0.029824890196323395
0.02934165485203266
0.028878064826130867
0.028433555737137794
0.028007542714476585
0.027599439024925232
0.02720862440764904
0.02683452144265175
0.026476513594388962
0.02613401971757412
0.025806443765759468
0.02549322135746479
0.025193793699145317
0.02

In [175]:
beta, mu = model(C_pairwise, T_pairwise)
print(beta.shape, mu.shape, X_pairwise.shape, Y_pairwise.shape)
Y_hat = beta.squeeze() * X_pairwise + mu
Y_hat.shape

torch.Size([400, 1]) torch.Size([400]) torch.Size([400]) torch.Size([400])


torch.Size([400])

In [176]:
Y_hat[:10], Y_pairwise[:10]

(tensor([ 0.2773, -0.1024,  0.1854, -0.2136,  0.2652, -0.1464,  0.2533, -0.3177,
          0.2887,  0.0444], grad_fn=<SliceBackward0>),
 tensor([ 0.1800, -0.1988,  0.1800, -0.1988,  0.4678, -0.3332,  0.4678, -0.3332,
          0.3164, -0.0254]))